This is the second assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Again, please insert to code to your ApacheCouchDB based Cloudant instance below using the "Insert Code" function of Watson Studio( you've done this in Assignment 1 before)

Done, just execute all cells one after the other and you are done - just note that in the last one you have to update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

In [1]:
# The code was removed by Watson Studio for sharing.

Let's create a SparkSession object and put the Cloudant credentials into it

In [2]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [3]:
df=spark.read.load('shake_classification', "com.cloudant.spark")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+-----------+-----+-----+-----+--------------------+--------------------+
|CLASS|   SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+-----------+-----+-----+-----+--------------------+--------------------+
|    0|sureshkonti| 0.01| 0.01| 0.01|102f1fe84b1b6477f...|1-f9f148dc923770a...|
|    0|sureshkonti| 0.01| 0.01| 0.01|102f1fe84b1b6477f...|1-f9f148dc923770a...|
|    0|sureshkonti|  0.0|  0.0|  0.0|102f1fe84b1b6477f...|1-2a25fd372080ae8...|
|    0|sureshkonti| 0.57| 0.57| 0.57|102f1fe84b1b6477f...|1-69af8055dbcb199...|
|    0|sureshkonti|-5.96|-5.96|-5.96|102f1fe84b1b6477f...|1-64565388a51cab7...|
|    0|sureshkonti|-2.68|-2.68|-2.68|102f1fe84b1b6477f...|1-7fb70d4e5d96e25...|
|    0|sureshkonti| 3.24| 3.24| 3.24|102f1fe84b1b6477f...|1-28f4f7f54b17567...|
|    0|sureshkonti|-2.21|-2.21|-2.21|102f1fe84b1b6477f...|1-4d791edb0cf57dc...|
|    0|sureshkonti|-4.28|-4.28|-4.28|102f1fe84b1b6477f...|1-d5ea8240fe4451c...|
|    0|sureshkonti|-3.97|-3.97|-3.97|102

Please create a VectorAssembler which consumed columns X, Y and Z and produces a column “features”


In [14]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"], outputCol="features")

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to either
1.	Rename the “CLASS” column to “label” or
2.	Specify the label-column correctly to be “CLASS”


In [22]:
from pyspark.ml.classification import LogisticRegression

#classifier = ###YOUR_CODE_GOES_HERE###"
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="CLASS", outputCol="label")

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
classifier = lr

Let’s train and evaluate…


In [23]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[indexer, vectorAssembler, classifier])

In [21]:
df.show()

+-----+-----------+-----+-----+-----+--------------------+--------------------+
|CLASS|   SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+-----------+-----+-----+-----+--------------------+--------------------+
|    0|sureshkonti| 0.01| 0.01| 0.01|102f1fe84b1b6477f...|1-f9f148dc923770a...|
|    0|sureshkonti| 0.01| 0.01| 0.01|102f1fe84b1b6477f...|1-f9f148dc923770a...|
|    0|sureshkonti|  0.0|  0.0|  0.0|102f1fe84b1b6477f...|1-2a25fd372080ae8...|
|    0|sureshkonti| 0.57| 0.57| 0.57|102f1fe84b1b6477f...|1-69af8055dbcb199...|
|    0|sureshkonti|-5.96|-5.96|-5.96|102f1fe84b1b6477f...|1-64565388a51cab7...|
|    0|sureshkonti|-2.68|-2.68|-2.68|102f1fe84b1b6477f...|1-7fb70d4e5d96e25...|
|    0|sureshkonti| 3.24| 3.24| 3.24|102f1fe84b1b6477f...|1-28f4f7f54b17567...|
|    0|sureshkonti|-2.21|-2.21|-2.21|102f1fe84b1b6477f...|1-4d791edb0cf57dc...|
|    0|sureshkonti|-4.28|-4.28|-4.28|102f1fe84b1b6477f...|1-d5ea8240fe4451c...|
|    0|sureshkonti|-3.97|-3.97|-3.97|102

In [24]:
model = pipeline.fit(df)

In [25]:
prediction = model.transform(df)

In [26]:
prediction.show()

+-----+-----------+-----+-----+-----+--------------------+--------------------+-----+-------------------+--------------------+--------------------+----------+
|CLASS|   SENSORID|    X|    Y|    Z|                 _id|                _rev|label|           features|       rawPrediction|         probability|prediction|
+-----+-----------+-----+-----+-----+--------------------+--------------------+-----+-------------------+--------------------+--------------------+----------+
|    0|sureshkonti| 0.01| 0.01| 0.01|102f1fe84b1b6477f...|1-f9f148dc923770a...|  0.0|   [0.01,0.01,0.01]|[2.00477437921908...|[0.88129744651483...|       0.0|
|    0|sureshkonti| 0.01| 0.01| 0.01|102f1fe84b1b6477f...|1-f9f148dc923770a...|  0.0|   [0.01,0.01,0.01]|[2.00477437921908...|[0.88129744651483...|       0.0|
|    0|sureshkonti|  0.0|  0.0|  0.0|102f1fe84b1b6477f...|1-2a25fd372080ae8...|  0.0|          (3,[],[])|[2.00477437921908...|[0.88129744651483...|       0.0|
|    0|sureshkonti| 0.57| 0.57| 0.57|102f1fe84

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("label")
    
binEval.evaluate(prediction) 

0.8812974465148378

If you are happy with the result (I’m happy with > 0.55) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. (0.55 means that you are performing better than random guesses)


In [28]:
!rm -Rf a2_m2.json

In [29]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [30]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2019-01-13 20:22:25--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2019-01-13 20:22:26 (24.2 MB/s) - ‘rklib.py’ saved [2289/2289]



In [31]:
!zip -r a2_m2.json.zip a2_m2.json

  adding: a2_m2.json/ (stored 0%)
  adding: a2_m2.json/_SUCCESS (stored 0%)
  adding: a2_m2.json/._SUCCESS.crc (stored 0%)
  adding: a2_m2.json/.part-00000-4732966f-1982-4f77-8ba0-1f654f141c6e.json.crc (stored 0%)
  adding: a2_m2.json/part-00000-4732966f-1982-4f77-8ba0-1f654f141c6e.json (deflated 93%)


In [32]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [33]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = "suresh.konti@yahoo.com"
secret = "duOtErs8lhSzjMcs"

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/usr/local/src/spark21master/spark/python/pyspark/ml/wrapper.py", line 76, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'StringIndexer' object has no attribute '_java_obj'


Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"LTL4F","id":"f_F-qCtuEei_fRLwaVDk3g~LTL4F~cg5SxRejEemu7RI8YeKm8g","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
